In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (100, 100))  # Resize images if needed
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


Epoch 1/10
8/8 [==============================] - 8s 802ms/step - loss: 0.6941 - accuracy: 0.5898 - val_loss: 0.6467 - val_accuracy: 0.4844
Epoch 2/10
8/8 [==============================] - 6s 728ms/step - loss: 0.5455 - accuracy: 0.7266 - val_loss: 0.8512 - val_accuracy: 0.4844
Epoch 3/10
8/8 [==============================] - 4s 549ms/step - loss: 0.4826 - accuracy: 0.7852 - val_loss: 0.4674 - val_accuracy: 0.7500
Epoch 4/10
8/8 [==============================] - 4s 558ms/step - loss: 0.3448 - accuracy: 0.8438 - val_loss: 0.3039 - val_accuracy: 0.8438
Epoch 5/10
8/8 [==============================] - 8s 1s/step - loss: 0.2485 - accuracy: 0.8906 - val_loss: 0.2253 - val_accuracy: 0.8594
Epoch 6/10
8/8 [==============================] - 5s 575ms/step - loss: 0.2031 - accuracy: 0.9102 - val_loss: 0.1594 - val_accuracy: 0.9062
Epoch 7/10
8/8 [==============================] - 5s 688ms/step - loss: 0.1457 - accuracy: 0.9531 - val_loss: 0.1144 - val_accuracy: 0.9688
Epoch 8/10
8/8 [=======

In [9]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to MobileNetV2 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [-1, 1] as required by MobileNetV2
X_train = X_train / 127.5 - 1.0
X_test = X_test / 127.5 - 1.0

# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
8/8 [==============================] - 26s 3s/step - loss: 0.4486 - accuracy: 0.7656 - val_loss: 0.1419 - val_accuracy: 0.9844
Epoch 2/10
8/8 [==============================] - 13s 2s/step - loss: 0.1029 - accuracy: 0.9570 - val_loss: 0.0522 - val_accuracy: 0.9844
Epoch 3/10
8/8 [==============================] - 10s 1s/step - loss: 0.0446 - accuracy: 0.9922 - val_loss: 0.0347 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.0212 - accuracy: 1.0000 - val_loss: 0.0324 - val_accuracy: 0.9844
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 0.9844
Epoch 6/10
8/8 [==============================] - 14s 2s/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.0220 - val_accuracy: 0.9844
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0

In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to ResNet50 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by ResNet50
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

Epoch 1/10
8/8 [==============================] - 74s 9s/step - loss: 0.7451 - accuracy: 0.5430 - val_loss: 0.7128 - val_accuracy: 0.5156
Epoch 2/10
8/8 [==============================] - 73s 9s/step - loss: 0.7025 - accuracy: 0.5039 - val_loss: 0.6814 - val_accuracy: 0.4844
Epoch 3/10
8/8 [==============================] - 73s 10s/step - loss: 0.6672 - accuracy: 0.6094 - val_loss: 0.6550 - val_accuracy: 0.5469
Epoch 4/10
8/8 [==============================] - 71s 9s/step - loss: 0.6470 - accuracy: 0.6172 - val_loss: 0.6290 - val_accuracy: 0.7031
Epoch 5/10
8/8 [==============================] - 63s 8s/step - loss: 0.6296 - accuracy: 0.7695 - val_loss: 0.6277 - val_accuracy: 0.7188
Epoch 6/10
8/8 [==============================] - 74s 10s/step - loss: 0.6154 - accuracy: 0.7500 - val_loss: 0.6114 - val_accuracy: 0.7500
Epoch 7/10
8/8 [==============================] - 70s 9s/step - loss: 0.5948 - accuracy: 0.7695 - val_loss: 0.5868 - val_accuracy: 0.7656
Epoch 8/10
8/8 [================

In [11]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (299, 299))  # Resize images to InceptionV3 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by InceptionV3
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained InceptionV3 model without the top classification layer
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

87910968/87910968 [==============================] - 4s 0us/step
Epoch 1/10
8/8 [==============================] - 91s 11s/step - loss: 0.4121 - accuracy: 0.7891 - val_loss: 0.1495 - val_accuracy: 0.9844
Epoch 2/10
8/8 [==============================] - 87s 11s/step - loss: 0.1220 - accuracy: 0.9609 - val_loss: 0.1062 - val_accuracy: 0.9531
Epoch 3/10
8/8 [==============================] - 84s 11s/step - loss: 0.0829 - accuracy: 0.9688 - val_loss: 0.0531 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 85s 11s/step - loss: 0.0429 - accuracy: 0.9844 - val_loss: 0.0482 - val_accuracy: 0.9844
Epoch 5/10
8/8 [==============================] - 83s 11s/step - loss: 0.0314 - accuracy: 0.9961 - val_loss: 0.0434 - val_accuracy: 0.9844
Epoch 6/10
8/8 [==============================] - 84s 11s/step - loss: 0.0273 - accuracy: 0.9961 - val_loss: 0.0368 - val_accuracy: 0.9844
Epoch 7/10
8/8 [==============================] - 86s 11s/step - loss: 0.0165 - accuracy: 1.0000 - va

In [12]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to EfficientNetB0 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by EfficientNetB0
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained EfficientNetB0 model without the top classification layer
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/10
8/8 [==============================] - 35s 4s/step - loss: 0.7179 - accuracy: 0.5195 - val_loss: 0.6963 - val_accuracy: 0.4844
Epoch 2/10
8/8 [==============================] - 25s 3s/step - loss: 0.7162 - accuracy: 0.5195 - val_loss: 0.6926 - val_accuracy: 0.5156
Epoch 3/10
8/8 [==============================] - 24s 3s/step - loss: 0.6987 - accuracy: 0.4922 - val_loss: 0.7000 - val_accuracy: 0.4844
Epoch 4/10
8/8 [==============================] - 30s 4s/step - loss: 0.6990 - accuracy: 0.5195 - val_loss: 0.6940 - val_accuracy: 0.4844
Epoch 5/10
8/8 [==============================] - 29s 4s/step - loss: 0.7010 - accuracy: 0.4609 - val_loss: 0.6969 - val_accuracy: 0.4844
Epoch 6/10
8/8 [==============================] - 29s 4s/step - loss: 0.7034 - accuracy: 0.5195 - val_loss: 0.6951 - val_accuracy: 0.4844
Epoch 7/10
8/8 [==============================] - 26s 3s/step - loss: 0.6949 - accuracy: 0.5273 - val_loss:

In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to DenseNet121 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by DenseNet121
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained DenseNet121 model without the top classification layer
base_model = DenseNet121(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

29084464/29084464 [==============================] - 2s 0us/step
Epoch 1/10
8/8 [==============================] - 66s 7s/step - loss: 0.4440 - accuracy: 0.7852 - val_loss: 0.1801 - val_accuracy: 0.9531
Epoch 2/10
8/8 [==============================] - 70s 8s/step - loss: 0.1198 - accuracy: 0.9805 - val_loss: 0.0764 - val_accuracy: 0.9844
Epoch 3/10
8/8 [==============================] - 63s 8s/step - loss: 0.0607 - accuracy: 0.9844 - val_loss: 0.0558 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 65s 9s/step - loss: 0.0396 - accuracy: 0.9883 - val_loss: 0.0442 - val_accuracy: 0.9844
Epoch 5/10
8/8 [==============================] - 56s 7s/step - loss: 0.0285 - accuracy: 0.9961 - val_loss: 0.0385 - val_accuracy: 0.9844
Epoch 6/10
8/8 [==============================] - 66s 9s/step - loss: 0.0223 - accuracy: 0.9961 - val_loss: 0.0351 - val_accuracy: 0.9844
Epoch 7/10
8/8 [==============================] - 65s 9s/step - loss: 0.0167 - accuracy: 1.0000 - val_loss:

In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to VGG16 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by VGG16
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
8/8 [==============================] - 205s 26s/step - loss: 0.6140 - accuracy: 0.7070 - val_loss: 0.5415 - val_accuracy: 0.9531
Epoch 2/10
8/8 [==============================] - 202s 26s/step - loss: 0.5175 - accuracy: 0.8633 - val_loss: 0.4581 - val_accuracy: 0.9375
Epoch 3/10
8/8 [==============================] - 243s 31s/step - loss: 0.4335 - accuracy: 0.9258 - val_loss: 0.3874 - val_accuracy: 0.9375
Epoch 4/10
8/8 [==============================] - 203s 26s/step - loss: 0.3575 - accuracy: 0.9531 - val_loss: 0.3382 - val_accuracy: 0.9375
Epoch 5/10
8/8 [==============================] - 203s 26s/step - loss: 0.3040 - accuracy: 0.9648 - val_loss: 0.2817 - val_accuracy: 0.9531
Epoch 6/10
8/8 [==============================] - 203s 26s/step - loss: 0.2597 - accuracy: 0.9648 - val_loss: 0.2527 - val_accuracy: 0.9688
Epoch 7/10
8/8 [==============================] - 202s 26s/step - loss: 0.2272 - accuracy: 0.96

In [15]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (299, 299))  # Resize images to Xception input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by Xception
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained Xception model without the top classification layer
base_model = Xception(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

83683744/83683744 [==============================] - 3s 0us/step
Epoch 1/10
8/8 [==============================] - 159s 20s/step - loss: 0.5736 - accuracy: 0.6602 - val_loss: 0.4535 - val_accuracy: 0.7500
Epoch 2/10
8/8 [==============================] - 151s 20s/step - loss: 0.2774 - accuracy: 0.9102 - val_loss: 0.2097 - val_accuracy: 0.9375
Epoch 3/10
8/8 [==============================] - 153s 20s/step - loss: 0.1559 - accuracy: 0.9492 - val_loss: 0.1662 - val_accuracy: 0.9688
Epoch 4/10
8/8 [==============================] - 154s 20s/step - loss: 0.0887 - accuracy: 0.9883 - val_loss: 0.1482 - val_accuracy: 0.9375
Epoch 5/10
8/8 [==============================] - 155s 20s/step - loss: 0.0699 - accuracy: 0.9727 - val_loss: 0.1251 - val_accuracy: 0.9531
Epoch 6/10
8/8 [==============================] - 155s 20s/step - loss: 0.0463 - accuracy: 0.9922 - val_loss: 0.1189 - val_accuracy: 0.9531
Epoch 7/10
8/8 [==============================] - 140s 18s/step - loss: 0.0401 - accuracy: 0.99

In [16]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to ResNet50V2 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by ResNet50V2
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained ResNet50V2 model without the top classification layer
base_model = ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

94668760/94668760 [==============================] - 4s 0us/step
Epoch 1/10
8/8 [==============================] - 57s 7s/step - loss: 0.4877 - accuracy: 0.7266 - val_loss: 0.1702 - val_accuracy: 0.9375
Epoch 2/10
8/8 [==============================] - 64s 8s/step - loss: 0.1470 - accuracy: 0.9414 - val_loss: 0.0719 - val_accuracy: 0.9688
Epoch 3/10
8/8 [==============================] - 70s 9s/step - loss: 0.0767 - accuracy: 0.9727 - val_loss: 0.0462 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 53s 7s/step - loss: 0.0337 - accuracy: 0.9961 - val_loss: 0.0321 - val_accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 64s 8s/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.0262 - val_accuracy: 1.0000
Epoch 6/10
8/8 [==============================] - 64s 8s/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 0.0220 - val_accuracy: 1.0000
Epoch 7/10
8/8 [==============================] - 55s 7s/step - loss: 0.0112 - accuracy: 1.0000 - val_loss:

In [17]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize images to MobileNetV3 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by MobileNetV3
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained MobileNetV3Small model without the top classification layer
base_model = MobileNetV3Small(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

4334752/4334752 [==============================] - 1s 0us/step
Epoch 1/10
8/8 [==============================] - 10s 644ms/step - loss: 0.7013 - accuracy: 0.5273 - val_loss: 0.6961 - val_accuracy: 0.4844
Epoch 2/10
8/8 [==============================] - 4s 510ms/step - loss: 0.6914 - accuracy: 0.4805 - val_loss: 0.6875 - val_accuracy: 0.7969
Epoch 3/10
8/8 [==============================] - 3s 357ms/step - loss: 0.6858 - accuracy: 0.6016 - val_loss: 0.6918 - val_accuracy: 0.4844
Epoch 4/10
8/8 [==============================] - 3s 367ms/step - loss: 0.6857 - accuracy: 0.5469 - val_loss: 0.6837 - val_accuracy: 0.4844
Epoch 5/10
8/8 [==============================] - 3s 356ms/step - loss: 0.6802 - accuracy: 0.5195 - val_loss: 0.6863 - val_accuracy: 0.4844
Epoch 6/10
8/8 [==============================] - 4s 596ms/step - loss: 0.6745 - accuracy: 0.5781 - val_loss: 0.6746 - val_accuracy: 0.5312
Epoch 7/10
8/8 [==============================] - 4s 450ms/step - loss: 0.6759 - accuracy: 0.609

In [18]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (300, 300))  # Resize images to EfficientNetB3 input size
            images.append(img)
            labels.append(1 if folder.endswith("_00") else 0)  # Label 1 for healthy, 0 for diseased
    return images, labels

# Load images from both folders
folder_00 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_00"
folder_01 = "/content/drive/MyDrive/Research Thread (Pankaj Bhowmik Sir)/Apple Dataset/_01"
images_00, labels_00 = load_images(folder_00)
images_01, labels_01 = load_images(folder_01)

# Concatenate images and labels
images = np.array(images_00 + images_01)
labels = np.array(labels_00 + labels_01)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1] as required by EfficientNetB3
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained EfficientNetB3 model without the top classification layer
base_model = EfficientNetB3(input_shape=(300, 300, 3), include_top=False, weights='imagenet')

# Freeze the convolutional base
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

43941136/43941136 [==============================] - 2s 0us/step
Epoch 1/10
8/8 [==============================] - 122s 14s/step - loss: 0.7733 - accuracy: 0.4805 - val_loss: 0.7246 - val_accuracy: 0.4844
Epoch 2/10
8/8 [==============================] - 98s 12s/step - loss: 0.6910 - accuracy: 0.5273 - val_loss: 0.6922 - val_accuracy: 0.5156
Epoch 3/10
8/8 [==============================] - 98s 12s/step - loss: 0.6922 - accuracy: 0.4922 - val_loss: 0.6847 - val_accuracy: 0.4844
Epoch 4/10
8/8 [==============================] - 97s 12s/step - loss: 0.6718 - accuracy: 0.5820 - val_loss: 0.6722 - val_accuracy: 0.6875
Epoch 5/10
8/8 [==============================] - 97s 13s/step - loss: 0.6626 - accuracy: 0.7031 - val_loss: 0.6636 - val_accuracy: 0.6875
Epoch 6/10
8/8 [==============================] - 98s 13s/step - loss: 0.6616 - accuracy: 0.6406 - val_loss: 0.6568 - val_accuracy: 0.7188
Epoch 7/10
8/8 [==============================] - 106s 14s/step - loss: 0.6606 - accuracy: 0.5820 - 